## Sentinel-2 vegetation mask based on NDVI and BSI

### Service Definition

In [1]:
service = dict([('title', 'Sentinel-2 vegetation mask based on NDVI and BSI'),
                ('abstract', 'Sentinel-2 vegetation mask based on NDVI and BSI'),
                ('id', 'ewf-s2-vegetation-mask')])

**Number of classes**

Number of modes, which will be used to generate class membership

In [2]:
n_classes = dict([('id', 'n_classes'),
                  ('value', '1=-1#0,2=0#0.1,3=0.1#0.2,4=0.2#1'),
                  ('title', 'Classes and ranges for NDVI'),
                  ('abstract', 'Classes and ranges for NDVI (class_1=min#max,class_2=min#max)'),
                  ('maxOccurs', '20')])

In [3]:
b_classes = dict([('id', 'b_classes'),
                  ('value', '1=-1#-0.2,2=-0.2#0.1,3=0.1#0.2,4=0.2#1'),
                  ('title', 'Classes and ranges for BSI'),
                  ('abstract', 'Classes and ranges for BSI (class_1=min#max,class_2=min#max)'),
                  ('maxOccurs', '20')])

In [4]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                   ('value', '0.3'),
                   ('title', 'NDVI threshold for the mask expression'),
                   ('abstract', 'NDVI threshold for the mask expression'),
                   ('maxOccurs', '1')])

In [5]:
bsi_threshold = dict([('id', 'bsi_threshold'),
                   ('value', '0'),
                   ('title', 'BSI threshold'),
                   ('abstract', 'BSI threshold'),
                   ('maxOccurs', '1')])

In [6]:
aoi = dict([('id', 'aoi'),
              ('title', 'Area of interest'),
              ('abstract', 'Area of interest'),
              ('value', '-70.5659,-13.0922,-69.1411,-12.4567')])

In [7]:
username = dict([('id', '_T2Username'),
              ('title', 'Ellip username'),
              ('abstract', 'Ellip username'),
              ('value', '')])

In [8]:
api_key = dict([('id', '_T2ApiKey'),
              ('title', 'Ellip API key for data pipeline'),
              ('abstract', 'Ellip API key for data pipeline'),
              ('value', '')])

### <a name="runtime">Runtime parameter definition

**Input reference**

This is the Sentinel-2 catalogue entry URLs

In [10]:
input_references = []

### Workflow

#### Import the packages required for processing the data

In [11]:
import os
import sys

sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
from helpers import *
%load_ext autoreload
%autoreload 2


sys.path.append('/opt/OTB/lib/python')
sys.path.append('/opt/OTB/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
import otbApplication
import gdal

gdal.UseExceptions()

In [12]:
product_metadata = get_product_metadata(input_references, 
                                        username['value'],
                                        api_key['value'])

In [13]:
product_metadata

,cc,enclosure,enddate,identifier,platform,self,startdate,track,wkt,geometry
0,62.384787,https://store.terradue.com/ard-s2-boa-reflecta...,2019-09-07 14:57:29.024,11D2917F71F9DA03998978645FFBD2135C9C70BF,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2019-09-07 14:57:29.024,39,"POLYGON((-70.835297389016 -12.7511412207275,-6...",POLYGON ((-70.83529738901601 -12.7511412207275...
1,99.962690,https://store.terradue.com/ard-s2-boa-reflecta...,2019-09-07 14:57:29.024,5A584E95852728AD1EDC1683B7211CEF272B8B1E,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2019-09-07 14:57:29.024,39,"POLYGON((-69.9181391041721 -12.7524314684136,-...","POLYGON ((-69.9181391041721 -12.7524314684136,..."
2,91.178170,https://store.terradue.com/ard-s2-boa-reflecta...,2019-09-07 14:57:29.024,6190DE5C56AD4AAAAF61290088773211B4626D10,S2B,https://catalog.terradue.com/ard-s2-boa-reflec...,2019-09-07 14:57:29.024,39,"POLYGON((-69.9212607569422 -13.6568683838856,-...","POLYGON ((-69.9212607569422 -13.6568683838856,..."


In [14]:
min_date = min(product_metadata['startdate'])
max_date = max(product_metadata['enddate'])

#### Area of interest

In [15]:
x_min, y_min, x_max, y_max = [float(c) for c in aoi['value'].split(',')]

In [16]:
x_min, y_min, x_max, y_max

(-70.5659, -13.0922, -69.1411, -12.4567)

#### Input vsi URLs

Use the Sentinel-2 ARD using GDAL Virtual File System

In [17]:
daily_products = []

for track in product_metadata.track.unique():

    print(track)
    input_references = list(product_metadata[product_metadata.track == track].self.values)
    
    print(input_references)

    output_name = str(track)
    
    pre_proces(input_references, 
               [x_min, y_min, x_max, y_max],
               ndvi_threshold['value'],
               bsi_threshold['value'],
               n_classes['value'], 
               b_classes['value'], 
               output_name, 
               username['value'], 
               api_key['value'])
    
    daily_products.append(output_name)

39
['https://catalog.terradue.com/ard-s2-boa-reflectances/search?format=atom&uid=11D2917F71F9DA03998978645FFBD2135C9C70BF', 'https://catalog.terradue.com/ard-s2-boa-reflectances/search?format=atom&uid=5A584E95852728AD1EDC1683B7211CEF272B8B1E', 'https://catalog.terradue.com/ard-s2-boa-reflectances/search?format=atom&uid=6190DE5C56AD4AAAAF61290088773211B4626D10']
1 B01
2 B02
3 B03
4 B04
5 B05
6 B06
7 B07
8 B08
9 B8A
10 B09
11 B11
12 B12
13 SCL
14 MSK_CLDPRB
15 MSK_SNWPRB


In [18]:
ndvi_classes = dict((int(k.strip()), v.strip().replace('#', ',')) for k,v in  
                             (item.split('=') for item in n_classes['value'].split(',')))

In [19]:
bsi_classes = dict((int(k.strip()), v.strip().replace('#', ',')) for k,v in  
                             (item.split('=') for item in b_classes['value'].split(',')))

In [20]:
prefix = ['NDVI_CLASS',
         'BSI_CLASS',
         'MASK',
         'VEGETATION_MASK']

for index, c in enumerate([ndvi_classes.keys(), bsi_classes.keys(), [0], [10, 20]]):
    
    expression = get_mosaic_expression(128, len(product_metadata.track.unique()), c)
    print expression
    date_format = '%Y%m%dT%H%m%S'
    
    output_name = '{0}_{1}_{2}.tif'.format(prefix[index], 
                                           min_date.strftime(date_format),
                                           max_date.strftime(date_format))
    
    daily_products = ['{}_{}.tif'.format(index, track) for track in product_metadata.track.unique()] 
    
    print output_name
    print daily_products
    
    
    BandMathX = otbApplication.Registry.CreateApplication('BandMathX')

    BandMathX.SetParameterStringList('il', daily_products)
    BandMathX.SetParameterString('out', 'temp_{0}'.format(output_name))
    BandMathX.SetParameterString('exp', expression)
    BandMathX.SetParameterOutputImagePixelType('out', otbApplication.ImagePixelType_uint8)

    BandMathX.ExecuteAndWriteOutput()

    cog('temp_{0}'.format(output_name),
        output_name)

    date_format = '%Y-%m-%dT%H:%m:%SZ'
        
    with open(output_name.replace('.tif', '.properties'), 'wb') as file:
        
        file.write('title={0} {1} {2}\n'.format(prefix[index], 
                                           min_date.strftime(date_format),
                                           max_date.strftime(date_format)))
        
        file.write('date={0}/{1}\n'.format(min_date.strftime(date_format),
                                           max_date.strftime(date_format)))
        
        file.write('geometry={0}'.format(get_image_wkt(output_name)))
        
    for prd in daily_products:
        os.remove(prd)
        os.remove(prd + '.aux.xml')

(im1b1 == 128) ? 128 : (im1b1 == 1) ? 1 : (im1b1 == 2) ? 2 : (im1b1 == 3) ? 3 : (im1b1 == 4) ? 4 : 255
NDVI_CLASS_20190907T140929_20190907T140929.tif
['0_39.tif']
(im1b1 == 128) ? 128 : (im1b1 == 1) ? 1 : (im1b1 == 2) ? 2 : (im1b1 == 3) ? 3 : (im1b1 == 4) ? 4 : 255
BSI_CLASS_20190907T140929_20190907T140929.tif
['1_39.tif']
(im1b1 == 128) ? 128 : (im1b1 == 0) ? 0 : 255
MASK_20190907T140929_20190907T140929.tif
['2_39.tif']
(im1b1 == 128) ? 128 : (im1b1 == 10) ? 10 : (im1b1 == 20) ? 20 : 255
VEGETATION_MASK_20190907T140929_20190907T140929.tif
['3_39.tif']


### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.